In [2]:
# Final validation of selected models and prediction on 3rd session for all subjects
import mne
import numpy as np
import pandas as pd
import sklearn
from sklearn import svm 
from sklearn.svm import SVC 
from sklearn.svm import NuSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier as VC
from sklearn.neighbors import KNeighborsClassifier as sknn
from sklearn.metrics import confusion_matrix as cm
from sklearn.ensemble import StackingClassifier as SC

from mne.externals.pymatreader import read_mat

from pyriemann.utils.distance import distance_riemann
from pyriemann.classification import MDM
from pyriemann.classification import KNearestNeighbor as knn
from pyriemann.classification import TSclassifier as tsc
from pyriemann.classification import FgMDM
from pyriemann.estimation import Covariances
from pyriemann.estimation import (XdawnCovariances, HankelCovariances,
                                  CospCovariances, ERPCovariances)
from pyriemann.spatialfilters import Xdawn, CSP
from pyriemann.tangentspace import TangentSpace


import xgboost as xgb

import matplotlib.pyplot as plt
%matplotlib qt

import pickle
import joblib
import os
import time
from collections import OrderedDict

mne.set_log_level('ERROR')

In [3]:
def load_dat(sub_n,session_n,epf=False):
    cwd = os.getcwd()
    data_path = cwd
    diff = ['MATBeasy', 'MATBmed', 'MATBdiff']

    epochs_data = []
    labels = []
    for lab_idx, level in enumerate(diff):
        sub = 'P{0:02d}'.format(sub_n)
        sess = f'S{session_n}'
        path = os.path.join(os.path.join(data_path, sub), sess) + f'/eeg/alldata_sbj{str(sub_n).zfill(2)}_sess{session_n}_{level}.set'
        # Read the epoched data with MNE
        epochs = mne.io.read_epochs_eeglab(path, verbose=False)
        # You could add some pre-processing here with MNE

        # Get the data and concatenante with others MATB levels
        tmp = epochs.get_data()
        epochs_data.extend(tmp)
        labels.extend([lab_idx]*len(tmp))
    epochs_datan = np.array(epochs_data)
    labels = np.array(labels)
    if epf ==False:
        return epochs_datan, labels
    if epf==True:
        return epochs_datan, labels, epochs_data
    
def load_test_dat(sub_n,session_n=3,epf=False):
    cwd = os.getcwd()
    data_path = cwd
    epochs_data = []

    sub = 'P{0:02d}'.format(sub_n)
    sess = f'S{session_n}'
    path = os.path.join(os.path.join(data_path, sub), sess) + f'/eeg/testset_sbj{str(sub_n).zfill(2)}_sess{session_n}.set'
    # Read the epoched data with MNE
    epochs = mne.io.read_epochs_eeglab(path, verbose=False)
    tmp = epochs.get_data()
    epochs_data.extend(tmp)
    epochs_datan = np.array(epochs_data)

    if epf ==False:
        return epochs_datan
    if epf==True:
        return epochs_datan, epochs_data

In [ ]:
# ina2 = make_pipeline(HankelCovariances(delays=[2, 4, 8, 12, 16], estimator='oas'),
#                     TangentSpace(metric='logeuclid'),
#                     NuSVC())
# ina2.fit(X2,y2)
# p1 = ina2.predict(X1)
# p2 = ina2.predict(X2)
# print("tsc-svc hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

# c = np.around(cm(y1,p1,normalize='true'),2)
# c

In [4]:
# For the performance evaluation. Concatenate the session 1 and 2, 
# do train-test split (stratify class and group i.e. session) 80-20
# Do CV on the training data - See mean accuracy
# But I don't think this is a good method if data from 2 different sessions may be there, tsupdate won't work properly

mdmc = make_pipeline(Covariances(estimator='oas'),MDM(n_jobs=-1))
Fgmdmc = make_pipeline(Covariances(estimator='oas'),FgMDM(n_jobs=-1))

tsck2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=sknn(n_neighbors = 14,n_jobs=-1,weights='distance')))
svock2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsck2)], voting='hard',n_jobs=-1)

tscs3 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=NuSVC()))
svocs3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs3)], voting='hard',n_jobs=-1)

tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto')))
svocs2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs2)], voting='hard',n_jobs=-1)

tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,reg_lambda = 5,n_jobs=-1)))
# tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto')))
sf1 = VC(estimators = [('tsc',tscs2),('tscx',tscx),('mdm',mdmc)],weights = [1,1,2], voting='hard',n_jobs=-1)

tscs3s = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=NuSVC(probability=True)))

## The submitted model
ms = VC(estimators = [('mdm',mdmc),('tscx',tscx),('NuSVC',tscs3s)],voting='soft',n_jobs=-1)

array_clfs = {}
array_clfs["sknn_hard"] = svock2
array_clfs["NuSVC_hard"] = svocs3
array_clfs["NuSVC_xgb_soft"] = ms
array_clfs["xgb_SVC_weighted_hard"] = sf1
array_clfs["SVC_hard"] = svocs2

In [10]:
# Load a subject, run all classifier, delete data variables, repeat

X_test,y_test = load_dat(1,1)
X_train,y_train = load_dat(1,2)
X_p = load_test_dat(1,3)
print(X_test.shape,y_test.shape,X_train.shape,y_train.shape,X_p.shape)

(447, 61, 500) (447,) (447, 61, 500) (447,) (447, 61, 500)


In [21]:
sub = 1
fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{list(array_clfs.keys())[0]}_{sub}.csv"
fname

'D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/sknn_hard_1.csv'

In [33]:
X_test,y_test = load_dat(1,1)
X_train,y_train = load_dat(1,2)
X_p = load_test_dat(1,3)

for key,clf in array_clfs.items():
    sub = 1
    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    clf.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = clf.predict(X_train)
    p_test = clf.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = clf.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)

model sknn_hard,Train time = 13.102108478546143, Predict time = 50.78337216377258, train acc = 1.0, test acc = 0.6554809843400448.
[[0.87 0.03 0.1 ]
 [0.05 0.85 0.11]
 [0.19 0.56 0.25]]
model NuSVC_hard,Train time = 11.35181736946106, Predict time = 51.6494517326355, train acc = 1.0, test acc = 0.6756152125279642.
[[0.84 0.07 0.09]
 [0.06 0.88 0.06]
 [0.16 0.53 0.31]]
model NuSVC_xgb_soft,Train time = 27.175652027130127, Predict time = 93.99026417732239, train acc = 1.0, test acc = 0.6621923937360179.
[[0.7  0.23 0.07]
 [0.09 0.66 0.26]
 [0.11 0.26 0.63]]
model xgb_SVC_weighted_hard,Train time = 25.153958082199097, Predict time = 98.27040338516235, train acc = 0.8747203579418344, test acc = 0.6599552572706935.
[[0.77 0.19 0.04]
 [0.16 0.62 0.21]
 [0.11 0.3  0.59]]
model SVC_hard,Train time = 11.288834571838379, Predict time = 50.26536464691162, train acc = 0.901565995525727, test acc = 0.7002237136465325.
[[0.88 0.03 0.09]
 [0.05 0.9  0.05]
 [0.12 0.56 0.32]]


In [34]:
import pickle
for key,clf in array_clfs.items():
    sub = 1
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))

model sknn_hard,Train time = 55.83969163894653, Predict time = 70.01193857192993, train acc = 1.0, test acc = 0.6554809843400448.
[[0.87 0.03 0.1 ]
 [0.05 0.85 0.11]
 [0.19 0.56 0.25]]
model NuSVC_hard,Train time = 13.053358793258667, Predict time = 55.336960554122925, train acc = 1.0, test acc = 0.6756152125279642.
[[0.84 0.07 0.09]
 [0.06 0.88 0.06]
 [0.16 0.53 0.31]]
model NuSVC_xgb_soft,Train time = 37.33266758918762, Predict time = 90.49095058441162, train acc = 1.0, test acc = 0.6599552572706935.
[[0.69 0.23 0.08]
 [0.09 0.66 0.25]
 [0.11 0.26 0.62]]
model xgb_SVC_weighted_hard,Train time = 46.777177572250366, Predict time = 91.1038875579834, train acc = 0.8747203579418344, test acc = 0.6599552572706935.
[[0.77 0.19 0.04]
 [0.16 0.62 0.21]
 [0.11 0.3  0.59]]


model SVC_hard,Train time = 20.383556127548218, Predict time = 49.5918607711792, train acc = 0.901565995525727, test acc = 0.7002237136465325.
[[0.88 0.03 0.09]
 [0.05 0.9  0.05]
 [0.12 0.56 0.32]]


In [28]:
from sklearn.utils.validation import check_is_fitted

a = check_is_fitted(array_clfs["sknn_hard"])
print(a)
a = check_is_fitted(svock2)
print(a)

None
None


In [36]:
sub = 2
X_test,y_test = load_dat(sub,1)
X_train,y_train = load_dat(sub,2)
X_p = load_test_dat(sub,3)
print(sub)
for key,clf in array_clfs.items():
    sub = 2
    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    clf.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = clf.predict(X_train)
    p_test = clf.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = clf.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)
    
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))
    

2
model sknn_hard,Train time = 10.802088737487793, Predict time = 56.60003852844238, train acc = 1.0, test acc = 0.5503355704697986.
[[0.77 0.01 0.22]
 [0.17 0.01 0.83]
 [0.11 0.01 0.87]]
model NuSVC_hard,Train time = 13.638078212738037, Predict time = 55.22952437400818, train acc = 1.0, test acc = 0.5704697986577181.
[[0.78 0.01 0.21]
 [0.15 0.01 0.85]
 [0.06 0.01 0.93]]
model NuSVC_xgb_soft,Train time = 23.57753300666809, Predict time = 94.15355610847473, train acc = 1.0, test acc = 0.5838926174496645.
[[0.81 0.03 0.15]
 [0.19 0.11 0.7 ]
 [0.11 0.05 0.83]]
model xgb_SVC_weighted_hard,Train time = 22.97852897644043, Predict time = 92.6718316078186, train acc = 0.7829977628635347, test acc = 0.5458612975391499.
[[0.79 0.05 0.16]
 [0.21 0.08 0.7 ]
 [0.15 0.08 0.77]]
model SVC_hard,Train time = 11.410377740859985, Predict time = 51.59993624687195, train acc = 0.8568232662192393, test acc = 0.5659955257270693.
[[0.79 0.01 0.21]
 [0.16 0.01 0.83]
 [0.08 0.01 0.91]]


In [37]:
sub = 3
X_test,y_test = load_dat(sub,1)
X_train,y_train = load_dat(sub,2)
X_p = load_test_dat(sub,3)
print(sub)
for key,clf in array_clfs.items():
    sub = 3
    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    clf.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = clf.predict(X_train)
    p_test = clf.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = clf.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)
    
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))
    

3
model sknn_hard,Train time = 10.142722129821777, Predict time = 51.68991661071777, train acc = 1.0, test acc = 0.34675615212527966.
[[0.06 0.94 0.  ]
 [0.06 0.94 0.  ]
 [0.04 0.92 0.04]]
model NuSVC_hard,Train time = 10.911060094833374, Predict time = 51.96002817153931, train acc = 0.970917225950783, test acc = 0.34675615212527966.
[[0.06 0.94 0.  ]
 [0.06 0.94 0.  ]
 [0.04 0.92 0.04]]
model NuSVC_xgb_soft,Train time = 24.340091466903687, Predict time = 98.04278469085693, train acc = 0.9686800894854586, test acc = 0.7494407158836689.
[[0.62 0.36 0.03]
 [0.19 0.68 0.13]
 [0.   0.05 0.95]]
model xgb_SVC_weighted_hard,Train time = 20.40107274055481, Predict time = 89.57851123809814, train acc = 0.912751677852349, test acc = 0.5078299776286354.
[[0.81 0.19 0.  ]
 [0.33 0.67 0.  ]
 [0.   0.96 0.04]]
model SVC_hard,Train time = 11.442517757415771, Predict time = 52.30019736289978, train acc = 0.9261744966442953, test acc = 0.34675615212527966.
[[0.06 0.94 0.  ]
 [0.06 0.94 0.  ]
 [0.04 0.9

In [38]:
sub = 4
X_test,y_test = load_dat(sub,1)
X_train,y_train = load_dat(sub,2)
X_p = load_test_dat(sub,3)
print(sub)
for key,clf in array_clfs.items():
    sub = 4
    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    clf.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = clf.predict(X_train)
    p_test = clf.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = clf.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)
    
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))
    

4
model sknn_hard,Train time = 10.191020011901855, Predict time = 50.723453521728516, train acc = 1.0, test acc = 0.43847874720357943.
[[0.74 0.19 0.07]
 [0.6  0.11 0.29]
 [0.48 0.05 0.46]]
model NuSVC_hard,Train time = 10.701445817947388, Predict time = 50.81006979942322, train acc = 1.0, test acc = 0.44742729306487694.
[[0.83 0.1  0.07]
 [0.64 0.02 0.34]
 [0.5  0.01 0.5 ]]
model NuSVC_xgb_soft,Train time = 21.51299500465393, Predict time = 87.06529498100281, train acc = 1.0, test acc = 0.35570469798657717.
[[0.03 0.7  0.28]
 [0.   0.23 0.77]
 [0.   0.19 0.81]]
model xgb_SVC_weighted_hard,Train time = 21.812941551208496, Predict time = 86.57501101493835, train acc = 0.8411633109619687, test acc = 0.39373601789709173.
[[0.03 0.9  0.07]
 [0.   0.66 0.34]
 [0.   0.5  0.5 ]]
model SVC_hard,Train time = 10.832484245300293, Predict time = 51.037747621536255, train acc = 0.8680089485458613, test acc = 0.34675615212527966.
[[0.19 0.73 0.07]
 [0.31 0.35 0.34]
 [0.33 0.17 0.5 ]]


In [39]:
sub = 5
X_test,y_test = load_dat(sub,1)
X_train,y_train = load_dat(sub,2)
X_p = load_test_dat(sub,3)
print(sub)
for key,clf in array_clfs.items():
    sub = 5
    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    clf.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = clf.predict(X_train)
    p_test = clf.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = clf.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)
    
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))
    

5
model sknn_hard,Train time = 11.408822774887085, Predict time = 63.38685083389282, train acc = 1.0, test acc = 0.5682326621923938.
[[0.81 0.03 0.16]
 [0.47 0.05 0.48]
 [0.15 0.01 0.84]]
model NuSVC_hard,Train time = 12.177934646606445, Predict time = 63.07461452484131, train acc = 0.9776286353467561, test acc = 0.5592841163310962.
[[0.81 0.03 0.17]
 [0.48 0.05 0.48]
 [0.17 0.01 0.83]]
model NuSVC_xgb_soft,Train time = 25.05189800262451, Predict time = 112.55902242660522, train acc = 0.9776286353467561, test acc = 0.6442953020134228.
[[0.8  0.11 0.09]
 [0.39 0.35 0.26]
 [0.15 0.06 0.79]]
model xgb_SVC_weighted_hard,Train time = 24.326865911483765, Predict time = 114.79849982261658, train acc = 0.6442953020134228, test acc = 0.5883668903803132.
[[0.83 0.05 0.12]
 [0.5  0.21 0.29]
 [0.21 0.07 0.72]]
model SVC_hard,Train time = 12.252467393875122, Predict time = 64.82497787475586, train acc = 0.7069351230425056, test acc = 0.5682326621923938.
[[0.8  0.03 0.17]
 [0.44 0.09 0.48]
 [0.16 0.

In [40]:
sub = 6
X_test,y_test = load_dat(sub,1)
X_train,y_train = load_dat(sub,2)
X_p = load_test_dat(sub,3)
print(sub)
for key,clf in array_clfs.items():
    sub = 6
    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    clf.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = clf.predict(X_train)
    p_test = clf.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = clf.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)
    
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))
    

6
model sknn_hard,Train time = 10.731504917144775, Predict time = 53.75531220436096, train acc = 1.0, test acc = 0.6062639821029083.
[[0.81 0.03 0.16]
 [0.22 0.21 0.57]
 [0.13 0.07 0.81]]
model NuSVC_hard,Train time = 11.640982866287231, Predict time = 53.52612376213074, train acc = 0.9798657718120806, test acc = 0.6062639821029083.
[[0.85 0.   0.15]
 [0.23 0.14 0.63]
 [0.14 0.03 0.83]]
model NuSVC_xgb_soft,Train time = 20.55870270729065, Predict time = 92.89141273498535, train acc = 0.9798657718120806, test acc = 0.5883668903803132.
[[0.82 0.03 0.15]
 [0.16 0.16 0.68]
 [0.11 0.1  0.79]]
model xgb_SVC_weighted_hard,Train time = 22.350396871566772, Predict time = 93.02756381034851, train acc = 0.8411633109619687, test acc = 0.5883668903803132.
[[0.78 0.05 0.17]
 [0.16 0.29 0.55]
 [0.15 0.15 0.7 ]]
model SVC_hard,Train time = 11.53057312965393, Predict time = 53.90191149711609, train acc = 0.9172259507829977, test acc = 0.5816554809843401.
[[0.83 0.01 0.16]
 [0.26 0.25 0.5 ]
 [0.25 0.08 

In [41]:
sub = 7
X_test,y_test = load_dat(sub,1)
X_train,y_train = load_dat(sub,2)
X_p = load_test_dat(sub,3)
print(sub)
for key,clf in array_clfs.items():
    sub = 7
    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    clf.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = clf.predict(X_train)
    p_test = clf.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = clf.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)
    
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))
  

7
model sknn_hard,Train time = 10.101146697998047, Predict time = 53.06065011024475, train acc = 1.0, test acc = 0.4541387024608501.
[[0.92 0.01 0.07]
 [0.62 0.11 0.26]
 [0.58 0.09 0.33]]
model NuSVC_hard,Train time = 10.892117738723755, Predict time = 53.61599636077881, train acc = 0.9865771812080537, test acc = 0.42058165548098436.
[[0.91 0.01 0.07]
 [0.73 0.06 0.21]
 [0.62 0.09 0.29]]
model NuSVC_xgb_soft,Train time = 23.075266361236572, Predict time = 92.45352077484131, train acc = 0.9821029082774049, test acc = 0.47874720357941836.
[[0.81 0.1  0.09]
 [0.31 0.17 0.52]
 [0.27 0.28 0.45]]
model xgb_SVC_weighted_hard,Train time = 23.102816104888916, Predict time = 92.66772651672363, train acc = 0.6331096196868009, test acc = 0.49217002237136465.
[[0.9  0.03 0.07]
 [0.48 0.25 0.28]
 [0.45 0.22 0.33]]
model SVC_hard,Train time = 10.943039417266846, Predict time = 54.119901180267334, train acc = 0.6778523489932886, test acc = 0.4988814317673378.
[[0.92 0.01 0.07]
 [0.49 0.25 0.26]
 [0.46

In [42]:
sub = 8
X_test,y_test = load_dat(sub,1)
X_train,y_train = load_dat(sub,2)
X_p = load_test_dat(sub,3)
print(sub)
for key,clf in array_clfs.items():
    sub = 8
    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    clf.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = clf.predict(X_train)
    p_test = clf.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = clf.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)
    
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))
    

8
model sknn_hard,Train time = 10.6277174949646, Predict time = 53.508241176605225, train acc = 1.0, test acc = 0.5883668903803132.
[[0.89 0.07 0.04]
 [0.47 0.21 0.32]
 [0.3  0.04 0.66]]
model NuSVC_hard,Train time = 12.01052975654602, Predict time = 53.799922704696655, train acc = 0.9977628635346756, test acc = 0.5995525727069351.
[[0.9  0.09 0.01]
 [0.42 0.31 0.27]
 [0.33 0.08 0.59]]
model NuSVC_xgb_soft,Train time = 23.59673833847046, Predict time = 92.65569281578064, train acc = 0.9977628635346756, test acc = 0.6733780760626398.
[[0.8  0.19 0.01]
 [0.03 0.49 0.48]
 [0.07 0.19 0.73]]
model xgb_SVC_weighted_hard,Train time = 23.834446907043457, Predict time = 92.56416606903076, train acc = 0.6890380313199105, test acc = 0.6577181208053692.
[[0.81 0.09 0.1 ]
 [0.07 0.39 0.54]
 [0.1  0.12 0.78]]
model SVC_hard,Train time = 11.6311354637146, Predict time = 53.72228002548218, train acc = 0.7606263982102909, test acc = 0.6286353467561522.
[[0.84 0.07 0.09]
 [0.21 0.25 0.54]
 [0.14 0.06 0.

In [43]:
sub = 9
X_test,y_test = load_dat(sub,1)
X_train,y_train = load_dat(sub,2)
X_p = load_test_dat(sub,3)
print(sub)
for key,clf in array_clfs.items():
    sub = 9
    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    clf.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = clf.predict(X_train)
    p_test = clf.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = clf.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)
    
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))
    

9
model sknn_hard,Train time = 11.151219367980957, Predict time = 58.0589017868042, train acc = 1.0, test acc = 0.4161073825503356.
[[0.32 0.68 0.  ]
 [0.09 0.91 0.  ]
 [0.01 0.97 0.02]]
model NuSVC_hard,Train time = 11.570015668869019, Predict time = 57.99423003196716, train acc = 0.9530201342281879, test acc = 0.4161073825503356.
[[0.32 0.68 0.  ]
 [0.09 0.91 0.  ]
 [0.01 0.97 0.02]]
model NuSVC_xgb_soft,Train time = 20.15811514854431, Predict time = 102.14719414710999, train acc = 0.9507829977628636, test acc = 0.5637583892617449.
[[0.6  0.4  0.01]
 [0.28 0.6  0.11]
 [0.02 0.49 0.49]]
model xgb_SVC_weighted_hard,Train time = 21.491820573806763, Predict time = 103.35075497627258, train acc = 0.8456375838926175, test acc = 0.4541387024608501.
[[0.68 0.32 0.  ]
 [0.32 0.68 0.  ]
 [0.02 0.98 0.  ]]
model SVC_hard,Train time = 11.754506587982178, Predict time = 58.101601362228394, train acc = 0.9485458612975392, test acc = 0.4161073825503356.
[[0.32 0.68 0.  ]
 [0.09 0.91 0.  ]
 [0.01 0.

In [44]:
sub = 10
X_test,y_test = load_dat(sub,1)
X_train,y_train = load_dat(sub,2)
X_p = load_test_dat(sub,3)
print(sub)
for key,clf in array_clfs.items():
    sub = 10
    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    clf.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = clf.predict(X_train)
    p_test = clf.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = clf.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)
    
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))
    

10
model sknn_hard,Train time = 9.099262475967407, Predict time = 42.58891725540161, train acc = 1.0, test acc = 0.5123042505592841.
[[0.   1.   0.  ]
 [0.   1.   0.  ]
 [0.01 0.46 0.54]]
model NuSVC_hard,Train time = 10.042159795761108, Predict time = 42.3510377407074, train acc = 0.9977628635346756, test acc = 0.5190156599552572.
[[0.   1.   0.  ]
 [0.   1.   0.  ]
 [0.   0.44 0.56]]
model NuSVC_xgb_soft,Train time = 18.051048517227173, Predict time = 70.19773292541504, train acc = 1.0, test acc = 0.5123042505592841.
[[0.02 0.98 0.  ]
 [0.03 0.97 0.  ]
 [0.03 0.43 0.54]]
model xgb_SVC_weighted_hard,Train time = 17.79991126060486, Predict time = 70.0085277557373, train acc = 0.9753914988814317, test acc = 0.43847874720357943.
[[0.02 0.98 0.  ]
 [0.02 0.98 0.  ]
 [0.   0.68 0.32]]
model SVC_hard,Train time = 9.745293378829956, Predict time = 42.45206689834595, train acc = 0.9776286353467561, test acc = 0.5190156599552572.
[[0.   1.   0.  ]
 [0.   1.   0.  ]
 [0.01 0.44 0.56]]


In [45]:
sub = 11
X_test,y_test = load_dat(sub,1)
X_train,y_train = load_dat(sub,2)
X_p = load_test_dat(sub,3)
print(sub)
for key,clf in array_clfs.items():
    sub = 11
    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    clf.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = clf.predict(X_train)
    p_test = clf.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = clf.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)
    
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))
    

11
model sknn_hard,Train time = 10.392640590667725, Predict time = 56.61525797843933, train acc = 1.0, test acc = 0.3333333333333333.
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
model NuSVC_hard,Train time = 11.21146821975708, Predict time = 56.46492862701416, train acc = 0.9463087248322147, test acc = 0.33557046979865773.
[[0.99 0.01 0.  ]
 [0.99 0.01 0.  ]
 [0.98 0.02 0.  ]]
model NuSVC_xgb_soft,Train time = 21.515706539154053, Predict time = 98.25250244140625, train acc = 0.9463087248322147, test acc = 0.5346756152125279.
[[0.77 0.23 0.01]
 [0.16 0.79 0.05]
 [0.05 0.9  0.05]]
model xgb_SVC_weighted_hard,Train time = 21.182105779647827, Predict time = 98.26710748672485, train acc = 0.8590604026845637, test acc = 0.5324384787472036.
[[0.77 0.23 0.  ]
 [0.17 0.83 0.  ]
 [0.05 0.95 0.  ]]
model SVC_hard,Train time = 11.400665998458862, Predict time = 56.4250807762146, train acc = 0.9284116331096197, test acc = 0.3333333333333333.
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [46]:
sub = 12
X_test,y_test = load_dat(sub,1)
X_train,y_train = load_dat(sub,2)
X_p = load_test_dat(sub,3)
print(sub)
for key,clf in array_clfs.items():
    sub = 12
    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    clf.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = clf.predict(X_train)
    p_test = clf.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = clf.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)
    
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))
    

12
model sknn_hard,Train time = 10.635932207107544, Predict time = 53.745107889175415, train acc = 1.0, test acc = 0.43400447427293065.
[[0.53 0.34 0.13]
 [0.4  0.52 0.07]
 [0.19 0.56 0.25]]
model NuSVC_hard,Train time = 11.382272481918335, Predict time = 53.92194676399231, train acc = 0.9955257270693513, test acc = 0.5167785234899329.
[[0.5  0.17 0.33]
 [0.3  0.58 0.11]
 [0.11 0.42 0.47]]
model NuSVC_xgb_soft,Train time = 28.693744897842407, Predict time = 92.57937145233154, train acc = 0.9977628635346756, test acc = 0.4966442953020134.
[[0.51 0.34 0.15]
 [0.34 0.57 0.09]
 [0.13 0.46 0.41]]
model xgb_SVC_weighted_hard,Train time = 28.42510986328125, Predict time = 93.70217537879944, train acc = 0.6129753914988815, test acc = 0.4519015659955257.
[[0.46 0.52 0.03]
 [0.26 0.72 0.03]
 [0.23 0.59 0.18]]
model SVC_hard,Train time = 11.379311323165894, Predict time = 54.249030351638794, train acc = 0.6442953020134228, test acc = 0.5279642058165548.
[[0.42 0.44 0.14]
 [0.22 0.66 0.11]
 [0.19 

In [47]:
sub = 13
X_test,y_test = load_dat(sub,1)
X_train,y_train = load_dat(sub,2)
X_p = load_test_dat(sub,3)
print(sub)
for key,clf in array_clfs.items():
    sub = 13
    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    clf.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = clf.predict(X_train)
    p_test = clf.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = clf.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)
    
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))
    

13
model sknn_hard,Train time = 11.50284743309021, Predict time = 52.046220541000366, train acc = 1.0, test acc = 0.33109619686800895.
[[0.46 0.04 0.5 ]
 [0.21 0.07 0.72]
 [0.34 0.2  0.46]]
model NuSVC_hard,Train time = 11.380378007888794, Predict time = 52.36738085746765, train acc = 0.9932885906040269, test acc = 0.37360178970917224.
[[0.67 0.05 0.28]
 [0.36 0.09 0.55]
 [0.4  0.24 0.36]]
model NuSVC_xgb_soft,Train time = 23.037548303604126, Predict time = 89.27395296096802, train acc = 0.9932885906040269, test acc = 0.4228187919463087.
[[0.87 0.07 0.07]
 [0.49 0.23 0.28]
 [0.46 0.37 0.17]]
model xgb_SVC_weighted_hard,Train time = 22.91073226928711, Predict time = 89.50749802589417, train acc = 0.6823266219239373, test acc = 0.36017897091722595.
[[0.67 0.06 0.27]
 [0.45 0.13 0.42]
 [0.47 0.25 0.28]]
model SVC_hard,Train time = 11.536175966262817, Predict time = 52.183273792266846, train acc = 0.7740492170022372, test acc = 0.3400447427293065.
[[0.54 0.05 0.4 ]
 [0.25 0.09 0.66]
 [0.37

In [48]:
sub = 14
X_test,y_test = load_dat(sub,1)
X_train,y_train = load_dat(sub,2)
X_p = load_test_dat(sub,3)
print(sub)
for key,clf in array_clfs.items():
    sub = 14
    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    clf.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = clf.predict(X_train)
    p_test = clf.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = clf.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)
    
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))
    

14
model sknn_hard,Train time = 10.733843564987183, Predict time = 56.404130697250366, train acc = 1.0, test acc = 0.4720357941834452.
[[0.58 0.04 0.38]
 [0.34 0.01 0.65]
 [0.17 0.   0.83]]
model NuSVC_hard,Train time = 11.94075083732605, Predict time = 57.117955684661865, train acc = 1.0, test acc = 0.44742729306487694.
[[0.64 0.05 0.31]
 [0.41 0.01 0.58]
 [0.31 0.   0.69]]
model NuSVC_xgb_soft,Train time = 20.419580459594727, Predict time = 98.82681083679199, train acc = 1.0, test acc = 0.37360178970917224.
[[0.6  0.35 0.05]
 [0.57 0.03 0.4 ]
 [0.4  0.12 0.48]]
model xgb_SVC_weighted_hard,Train time = 21.002515077590942, Predict time = 98.75317192077637, train acc = 0.8031319910514542, test acc = 0.43400447427293065.
[[0.72 0.19 0.09]
 [0.52 0.03 0.46]
 [0.42 0.03 0.56]]
model SVC_hard,Train time = 11.757803440093994, Predict time = 56.85169863700867, train acc = 0.8970917225950783, test acc = 0.4272930648769575.
[[0.65 0.04 0.31]
 [0.42 0.01 0.58]
 [0.38 0.   0.62]]


In [49]:
sub = 15
X_test,y_test = load_dat(sub,1)
X_train,y_train = load_dat(sub,2)
X_p = load_test_dat(sub,3)
print(sub)
for key,clf in array_clfs.items():
    sub = 15
    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    clf.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = clf.predict(X_train)
    p_test = clf.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = clf.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)
    
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))
    

15
model sknn_hard,Train time = 10.140229225158691, Predict time = 48.88445591926575, train acc = 1.0, test acc = 0.4854586129753915.
[[0.73 0.25 0.02]
 [0.35 0.23 0.42]
 [0.36 0.14 0.5 ]]
model NuSVC_hard,Train time = 10.730815887451172, Predict time = 49.31906270980835, train acc = 1.0, test acc = 0.5055928411633109.
[[0.81 0.17 0.01]
 [0.4  0.2  0.4 ]
 [0.35 0.15 0.5 ]]
model NuSVC_xgb_soft,Train time = 23.741156816482544, Predict time = 83.92739486694336, train acc = 0.9977628635346756, test acc = 0.44966442953020136.
[[0.67 0.32 0.01]
 [0.38 0.26 0.37]
 [0.34 0.24 0.42]]
model xgb_SVC_weighted_hard,Train time = 23.543327569961548, Predict time = 84.55523133277893, train acc = 0.7360178970917226, test acc = 0.4161073825503356.
[[0.6  0.39 0.01]
 [0.39 0.36 0.26]
 [0.37 0.34 0.29]]
model SVC_hard,Train time = 10.82939338684082, Predict time = 49.89279389381409, train acc = 0.8165548098434005, test acc = 0.4272930648769575.
[[0.6  0.39 0.01]
 [0.39 0.33 0.28]
 [0.37 0.28 0.36]]


In [8]:
elvl2_s = VC(estimators = [("NuSVC_hard",svocs3),("NuSVC_xgb_soft",ms)],voting='soft',n_jobs=-1)
# elvl2_h = VC(estimators = [("NuSVC_xgb_soft",ms),("NuSVC_hard",svocs3)],voting='hard',n_jobs=-1)

for sub in range(1,16):
    key = "lvl2_soft"
    X_test,y_test = load_dat(sub,1)
    X_train,y_train = load_dat(sub,2)
    X_p = load_test_dat(sub,3)
    print(sub)

    fname = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.csv"
    
    start = time.time()
    elvl2_s.fit(X_train,y_train)
    end = time.time()
    t1 = end-start
    
    start = time.time()
    p_train = elvl2_s.predict(X_train)
    p_test = elvl2_s.predict(X_test)
    end = time.time()
    t2 = end-start
    
    c = np.around(cm(y_test,p_test,normalize='true'),2)
    
    p_pred = elvl2_s.predict(X_p)
    pd.DataFrame(p_pred).to_csv(fname)
    
    print(f'model {key},Train time = {t1}, Predict time = {t2}, train acc = {accuracy_score(y_train,p_train)}, test acc = {accuracy_score(y_test,p_test)}.')
    print(c)
    
    filename = f"D:/Softwares/AnacondaWorkDir/Neuroergo_Hack/Predictions/{key}_{sub}.pickle"
    pickle.dump(clf, open(filename, 'wb'))
    

1


AttributeError: predict_proba is not available when voting='hard'

In [ ]:
tsck2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=sknn(n_neighbors = 14,n_jobs=-1,weights='distance')))
svock2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsck2)], voting='hard',n_jobs=-1)
svock2.fit(X2,y2)
pk1 = svock2.predict(X1)
pk2 = svock2.predict(X2)
print("svock2 hard train = ",accuracy_score(y2,pk2)," test = ",accuracy_score(y1,pk1))
ckx = np.around(cm(y1,pk1,normalize='true'),2)
ckx

In [ ]:
tscs3 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=NuSVC()))
svocs3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs3)], voting='hard',n_jobs=-1)
svocs3.fit(X2,y2)
pn1 = svocs3.predict(X1)
pn2 = svocs3.predict(X2)
print("svocs3 hard train = ",accuracy_score(y2,pn2)," test = ",accuracy_score(y1,pn1))
cnx = np.around(cm(y1,pn1,normalize='true'),2)
cnx

In [ ]:
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto')))
svocs2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs2)], voting='hard',n_jobs=-1)
svocs2.fit(X2,y2)
ps1 = svocs2.predict(X1)
ps2 = svocs2.predict(X2)
print("svocs2 hard train = ",accuracy_score(y2,ps2)," test = ",accuracy_score(y1,ps1))
csx = np.around(cm(y1,ps1,normalize='true'),2)
csx

In [ ]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,reg_lambda = 5,n_jobs=-1)))
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto')))
mdmc = make_pipeline(Covariances(estimator='oas'),MDM(n_jobs=-1))
sf1 = VC(estimators = [('tsc',tscs2),('tscx',tscx),('mdm',mdmc)],weights = [1,1,2], voting='hard',n_jobs=-1)
sf1.fit(X2,y2)
p1 = sf1.predict(X1)
p2 = sf1.predict(X2)
print("sf1 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
cf = np.around(cm(y1,p1,normalize='true'),2)
cf

In [ ]:
tscs3 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=NuSVC(probability=True)))
ms = VC(estimators = [('mdm',mdmc),('tscx',tscx),('NuSVC',tscs3)],voting='soft',n_jobs=-1)
ms.fit(X2,y2)
p1 = ms.predict(X1)
p2 = ms.predict(X2)
print("ms hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

cf = np.around(cm(y1,p1,normalize='true'),2)
cf